In [64]:
from bs4 import BeautifulSoup
from requests import get
%config Completer.use_jedi = False

In [28]:
url = 'https://www.bsc.edu/academics/computing/index.html'

response = get(url)
soup = BeautifulSoup(response.text)

In [73]:
from selenium import webdriver
import chromedriver_autoinstaller
import os
import re
from time import sleep

def create_driver(headless=True):
    driver = chromedriver_autoinstaller.install(cwd=True)
    chrome_options = webdriver.ChromeOptions()     
    prefs = {'download.default_directory' : os.getcwd()}
    chrome_options.add_experimental_option('prefs', prefs)
    if headless:
        chrome_options.add_argument("--headless")

    driver = webdriver.Chrome(driver, 
                             chrome_options = chrome_options)
    return driver

In [37]:
driver = create_driver()

/Users/joel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


# Faculty

In [50]:
root = 'https://www.bsc.edu/academics/'
url = root + 'index.html'
driver.get(url)

In [60]:
soup = BeautifulSoup(driver.page_source)
div = soup.find_all('h2', text=re.compile('Academic Departments'))[0].parent
a_tags = div.find_all('a')

In [71]:
department_links = {a.text: root + a.attrs['href'] for a in a_tags}

In [72]:
department_links

{'Applied Computer Science': 'https://www.bsc.edu/academics/computing/index.html',
 'Mathematics': 'https://www.bsc.edu/academics/math/index.html',
 'Art & Art History': 'https://www.bsc.edu/academics/art/index.html',
 'Media And Film Studies': 'https://www.bsc.edu/academics/media/index.html',
 'Biology': 'https://www.bsc.edu/academics/biology/index.html',
 'Music': 'https://www.bsc.edu/academics/music/index.html',
 'Business': 'https://www.bsc.edu/academics/business/index.html',
 'Philosophy': 'https://www.bsc.edu/academics/philosophy/index.html',
 'Chemistry': 'https://www.bsc.edu/academics/chem-phy/index.html',
 'Political Science': 'https://www.bsc.edu/academics/polysci-econ-soc/polisci.html',
 'Classics': 'https://www.bsc.edu/academics/greek-roman/index.html',
 'Psychology': 'https://www.bsc.edu/academics/psychology/index.html',
 'Economics': 'https://www.bsc.edu/academics/polysci-econ-soc/economics.html',
 'Physics': 'https://www.bsc.edu/academics/chem-phy/index.html',
 'Educatio

In [74]:
department_html = {}

for department in department_links:
    link = department_links[department]
    driver.get(link)
    department_html[department] = driver.page_source
    sleep(3)

In [82]:
department_soup = {}

for department in department_html:
    soup = BeautifulSoup(department_html[department])
    department_soup[department] = soup

In [570]:
def get_faculty_divs(program_soup):

    dropdown = program_soup.find('section', {'class': 'accordion'})

    # faculty_drop = dropdown.find('h2', text=re.compile('STAFF|FACULTY|Faculty|Staff')).find_previous('li')

    i_tags = dropdown.find_all('i')
    for tag in i_tags:
        if re.findall('STAFF|FACULTY|Faculty|Staff', tag.find_next('h2').text):
            faculty_drop = tag.find_previous('li')

    faculty = faculty_drop.find('div', {'class': 'content'}).find_all('p')

    return faculty

def parse_faculty_no_link(p_tag):
    phone = p_tag.find(text = re.compile('\(.+\) \d{3}-\d{4}|\d{3}-\d{3}-\d{4}'))
    if phone:
        phone = phone.strip()
    email = p_tag.find(text = re.compile('\@bsc\.edu'))
    return phone, email

def scrape_faculty_data(faculty_divs):
    data = []
    root = 'https://www.bsc.edu/academics/'
    for person in faculty_divs:

        default = {"name": None,
                   "office": None,
                  "phone": None,
                  "email": None,
                  "classes": None}
        name = list(person.strings)
        if not name:
            continue
        if len(name[0]) > 150:
            continue
        if not name[0].strip():
            try:
                name = name[1]
            except:
                continue
        else:
            name = name[0]
        default['name'] = name

        if person.find('a'):
            if person.find('a').attrs['href'][:2] == '..':
                href = person.find('a').attrs['href']
                faculty_url = root + href.replace('../', '')
                driver.get(faculty_url)
                faculty_html = driver.page_source
                faculty_soup = BeautifulSoup(faculty_html)
                try:
                    office = faculty_soup.find(text = re.compile("Office:"))
                    office = office.find_next('p').text
                    default['office'] = office
                    phone = faculty_soup.find(text = re.compile("Office Phone:"))
                    phone = phone.replace("Office Phone:", '').strip()
                    default['phone'] = phone
                    email = faculty_soup.find(text=re.compile("E-mail:"))
                    email = email.find_next('a').attrs['href'].replace('mailto:', '')
                    default['email'] = email
                    classes = faculty_soup.find(text=re.compile('Courses Taught:'))
                    classes = classes.find_next('blockquote').find_all('strong')
                    classes = [x.text for x in classes]
                    default['classes'] = classes
                except:
                    pass
            else:
                phone, email = parse_faculty_no_link(person)
                default['phone'] = phone
                default['email'] = email
        data.append(default)
        
    return data

In [571]:
faculty_data = {}

for department in department_soup:
    print(department)
    soup = department_soup[department]
    faculty_divs = get_faculty_divs(soup)
    data = scrape_faculty_data(faculty_divs)
    faculty_data[department] = data

Applied Computer Science
Mathematics
Art & Art History
Media And Film Studies
Biology
Music
Business
Philosophy
Chemistry
Political Science
Classics
Psychology
Economics
Physics
Education
Religion
English
Sociology
Foreign Languages
Theatre
History
Urban Environmental Studies


In [582]:
def create_driver(headless=True):
    driver = chromedriver_autoinstaller.install(cwd=True)
    chrome_options = webdriver.ChromeOptions()     
    prefs = {'download.default_directory' : os.getcwd()}
    chrome_options.add_experimental_option('prefs', prefs)
    if headless:
        chrome_options.add_argument("--headless")

    driver = webdriver.Chrome(driver, 
                             chrome_options = chrome_options)
    return driver

def get_department_links(driver):
    root = 'https://www.bsc.edu/academics/'
    url = root + 'index.html'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    div = soup.find_all('h2', text=re.compile('Academic Departments'))[0].parent
    a_tags = div.find_all('a')
    department_links = {a.text: root + a.attrs['href'] for a in a_tags}
    
    return department_links

def get_department_soup(driver, links):

    driver.get(link)
    html = driver.page_source
   
    return BeautifulSoup(html)




def get_faculty_divs(program_soup):

    dropdown = program_soup.find('section', {'class': 'accordion'})

    # faculty_drop = dropdown.find('h2', text=re.compile('STAFF|FACULTY|Faculty|Staff')).find_previous('li')

    i_tags = dropdown.find_all('i')
    for tag in i_tags:
        if re.findall('STAFF|FACULTY|Faculty|Staff', tag.find_next('h2').text):
            faculty_drop = tag.find_previous('li')

    faculty = faculty_drop.find('div', {'class': 'content'}).find_all('p')

    return faculty


def scrape_faculty_data(driver, faculty_divs):
    data = []
    root = 'https://www.bsc.edu/academics/'
    for person in faculty_divs:

        default = {"name": None,
                   "office": None,
                  "phone": None,
                  "email": None,
                  "classes": None}
        name = list(person.strings)
        if not name:
            continue
        if len(name[0]) > 150:
            continue
        if not name[0].strip():
            try:
                name = name[1]
            except:
                continue
        else:
            name = name[0]
        default['name'] = name

        if person.find('a'):
            if person.find('a').attrs['href'][:2] == '..':
                href = person.find('a').attrs['href']
                faculty_url = root + href.replace('../', '')
                driver.get(faculty_url)
                faculty_html = driver.page_source
                faculty_soup = BeautifulSoup(faculty_html)
                try:
                    office = faculty_soup.find(text = re.compile("Office:"))
                    office = office.find_next('p').text
                    default['office'] = office
                    phone = faculty_soup.find(text = re.compile("Office Phone:"))
                    phone = phone.replace("Office Phone:", '').strip()
                    default['phone'] = phone
                    email = faculty_soup.find(text=re.compile("E-mail:"))
                    email = email.find_next('a').attrs['href'].replace('mailto:', '')
                    default['email'] = email
                    classes = faculty_soup.find(text=re.compile('Courses Taught:'))
                    classes = classes.find_next('blockquote').find_all('strong')
                    classes = [x.text for x in classes]
                    default['classes'] = classes
                except:
                    pass
            else:
                phone, email = parse_faculty_no_link(person)
                default['phone'] = phone
                default['email'] = email
        data.append(default)
        
    return data

def parse_faculty_no_link(p_tag):
    phone = p_tag.find(text = re.compile('\(.+\) \d{3}-\d{4}|\d{3}-\d{3}-\d{4}'))
    if phone:
        phone = phone.strip()
    email = p_tag.find(text = re.compile('\@bsc\.edu'))
    return phone, email

def scrape_faculty():
    df = pd.DataFrame()
    driver = create_driver()
    program_links = get_department_links(driver)

    for program in program_links:
        print(f'Scraping data for {program}')
        link = program_links[program]
        soup = get_department_soup(driver, link)
        divs = get_faculty_divs(soup)
        data = scrape_faculty_data(driver, divs)
        frame = pd.DataFrame(data)
        frame['program'] = program
        df = df.append(frame)
    
    return df

/Users/joel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


In [583]:
df = scrape_faculty()

/Users/joel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


Scraping data for Applied Computer Science
Scraping data for Mathematics
Scraping data for Art & Art History
Scraping data for Media And Film Studies
Scraping data for Biology
Scraping data for Music
Scraping data for Business
Scraping data for Philosophy
Scraping data for Chemistry
Scraping data for Political Science
Scraping data for Classics
Scraping data for Psychology
Scraping data for Economics
Scraping data for Physics
Scraping data for Education
Scraping data for Religion
Scraping data for English
Scraping data for Sociology
Scraping data for Foreign Languages
Scraping data for Theatre
Scraping data for History
Scraping data for Urban Environmental Studies


In [585]:
df.dropna(subset=['email'])

,name,office,phone,email,classes,program
1,William G. Holt,313 Harbert,(205) 226-4834,wholt@bsc.edu,[UES 110 Sustainability: Southern Cities & The...,Applied Computer Science
2,Jane Archer,Berte Humanities Building 324,(205) 226-7838,jarcher@bsc.edu,[],Applied Computer Science
3,Larry Brasher,None,None,lbrasher@bsc.edu,None,Applied Computer Science
5,Scot Duncan,Stephens Science Center 248,(205) 226-4777,sduncan@bsc.edu,[],Applied Computer Science
6,Andrew Gannon,Stephens Science Center 230,(205) 226-4899,agannon@bsc.edu,"[BI 115 Organismal Biology (1), BI 232 Inverte...",Applied Computer Science
...,...,...,...,...,...,...
8,Megan Gibbons,Stephens Science Center 236,(205) 226-4874,mgibbons@bsc.edu,"[BI 115 Organismal Biology (1), BI 225 Evoluti...",Urban Environmental Studies
9,Bill Myers,HC 222,(205) 226-4868,bmyers@bsc.edu,[],Urban Environmental Studies
10,Rebekah Parker,None,None,rpparker@bsc.edu,None,Urban Environmental Studies
11,Kathleen Greer Rossmann,Harbert 309,(205) 226-4603,KRossman@bsc.edu,[],Urban Environmental Studies


# Senate

In [92]:
url = 'https://en.wikipedia.org/wiki/List_of_current_United_States_senators'
response = get(url)
soup = BeautifulSoup(response.text)

In [96]:
table = soup.find('table', {'id':'senators'})

In [140]:
senate = pd.read_html(str(table))[0]
senate.head(2)

,State,Portrait,Senator,Party,Party.1,Born,Occupation(s),Previous electiveoffice(s),Education,Assumed office,Term up,Residence
0,Alabama,NaN,Richard Shelby,NaN,Republican[2],(age 87),Lawyer,U.S. HouseAlabama Senate,University of Alabama Birmingham School of Law...,"January 3, 1987",2022,Tuscaloosa[3]
1,Alabama,NaN,Tommy Tuberville,NaN,Republican,(age 66),"College football coachPartner, investment mana...",None,Southern Arkansas University,"January 3, 2021",2026,Auburn


## A better scrape:

In [156]:
def scrape_senate_table():
    
    # Connect to the wikipedia page
    response = requests.get('https://en.wikipedia.org/wiki/List_of_current_United_States_senators')
    # Collect the html from the page
    html = response.text
    # Parse the html with BeautifulSoup
    soup = BeautifulSoup(html)
    # Find the senators table
    table = soup.find('table', {'id': 'senators'})
    
    # The first row in the table contains the column names.
    # Isolate the first row, then final all of the column tags.
    columns = table.find('tr').find_all('th')
    # Collect the row tags for the entire dataset
    rows = table.find_all('tr')[1:]
    # Create an empty list to append the row data to
    senate_data = []
    # Create some cleaning functions for text data
    remove_new_line = lambda x: x.replace('\n', '')
    split_new_line = lambda x: x.split('\n')
    
    # Loop over each row
    for row in rows:
        # find all td tags from the row
        td_tags = row.find_all('td')
        # ===============================================================================
        # The senators table merges the cell for `state name` so it spans both 
        # senators from a state. 
        # When parsing the html, the state name only appears for the senator
        # that appears first in the table, and the second appearing senator has 
        # one less td tag.
        # Because of this we need to check the length of the td tags
        # If there is one less tag, we add  the state name from the previous iteration
        # To the beginning of the list of tags
        # ===============================================================================
        # Check if the list of tags has the full number of columns
        if len(td_tags) == len(columns):
            # If it does, store the first element in the list of tags
            # to a variable called previous_element
            previous_element = td_tags[0]
        # If the list of tags does not have the full number of columns
        # insert the previous_element variable at the beginning of the list of tags
        else:
            td_tags.insert(0, previous_element)
        
        # ===============================================================================
        #                              Parse the row data
        # ===============================================================================
        # Collect the state name
        state = remove_new_line(td_tags[0].text)
        # Collect the image url
        image = td_tags[1].find('img').attrs['src']
        # Collect the name of the senator
        name = remove_new_line(row.find('th').text)
        # Collect the css color string for the political party
        party_color = td_tags[2].attrs['style'].split(':')[-1]
        # Collect the party name
        party_name = remove_new_line(td_tags[3].text)
        # Collect the date of birth
        dob = ' '.join(remove_new_line(td_tags[4]\
                                       .text)\
                                       .strip()\
                                       .split(' ')[1:4])
        # Collect the occupation
        occupation = split_new_line(BeautifulSoup(str(td_tags[5])\
                                                  .replace('<br/>', '\n'))\
                                                  .td\
                                                  .text)
        # If only one occupation is present
        # Pull that occupation out of the list
        # And return a single string
        if occupation[1] == '':
            occupation = occupation[0]
        # Collect the previous office
        previous_office = split_new_line(BeautifulSoup(str(td_tags[6])\
                                                       .replace('<br/>', '\n'))\
                                                       .td\
                                                      .text)
        # If only one previous office is present
        # Pull that value out of the list
        # And return a single string
        if previous_office[1] == '':
            previous_office = previous_office[0]
        # Collect assumed office
        assumed_office = remove_new_line(td_tags[7].text)
        # Collect the end of their term
        term_up = remove_new_line(td_tags[8].text)
        # Collect their residence
        residence = split_new_line(td_tags[9].text)
        
        # If only one residence is present
        # Pull that value out of the list
        # And return a single string
        if residence[1] == '':
            residence = residence[0]
            # Many of the residences have a 
            # link pointing to additional information
            # We do not need this
            if '[' in residence:
                residence = residence[:-3]
        # Create data dictionary
        collected = {'state': state, 'name': name,'dob': dob, 'party': party_name,
                    'party_color': party_color,
                    'occupation': occupation,
                    'previous_office': previous_office,
                    'assumed_office': assumed_office,
                    'term_up': term_up, 'residence': residence,
                    'portrait': image}
        # Append dictionary to the senate_data list
        senate_data.append(collected)
        
    # Once data from all rows has been collected
    # return the data as a pandas dataframe
    return pd.DataFrame(senate_data)

In [157]:
scrape_senate_table()

,state,name,dob,party,party_color,occupation,previous_office,assumed_office,term_up,residence,portrait
0,Alabama,Richard Shelby,"May 6, 1934",Republican[2],#E81B23,Lawyer,"[U.S. House, Alabama Senate, ]",University of AlabamaBirmingham School of LawU...,"January 3, 1987",2022,//upload.wikimedia.org/wikipedia/commons/thumb...
1,Alabama,Tommy Tuberville,"September 18, 1954",Republican,#E81B23,"[College football coach, Partner, investment m...",None,Southern Arkansas University,"January 3, 2021",2026,//upload.wikimedia.org/wikipedia/commons/thumb...
2,Alaska,Lisa Murkowski,"May 22, 1957",Republican,#E81B23,Lawyer,Alaska House of Representatives,Georgetown UniversityWillamette University Col...,"December 20, 2002[d]",2022,//upload.wikimedia.org/wikipedia/commons/thumb...
3,Alaska,Dan Sullivan,"November 13, 1964",Republican,#E81B23,"[U.S. Marine Corps officer, Lawyer, Assistant ...",Alaska Attorney General,Culver Military AcademyHarvard UniversityGeorg...,"January 3, 2015",2026,//upload.wikimedia.org/wikipedia/commons/thumb...
4,Arizona,Kyrsten Sinema,"July 12, 1976",Democratic,#3333FF,"[Social worker, Political activist, Lawyer, Co...","[U.S. House, Arizona Senate, Arizona House of ...",Brigham Young University,"January 3, 2019",2024,//upload.wikimedia.org/wikipedia/commons/thumb...
...,...,...,...,...,...,...,...,...,...,...,...
95,West Virginia,Shelley Moore Capito,"November 26, 1953",Republican,#E81B23,"[College career counselor, Director, state Boa...","[U.S. House, West Virginia House of Delegates, ]",Duke UniversityUniversity of Virginia,"January 3, 2015",2026,//upload.wikimedia.org/wikipedia/commons/thumb...
96,Wisconsin,Ron Johnson,"April 8, 1955",Republican,#E81B23,"[Accountant, Corporate executive, ]",None,University of Minnesota,"January 3, 2011",2022,//upload.wikimedia.org/wikipedia/commons/thumb...
97,Wisconsin,Tammy Baldwin,"February 11, 1962",Democratic,#3333FF,Lawyer,"[U.S. House, Wisconsin Assembly, Dane County, ...",Smith CollegeUniversity of Wisconsin,"January 3, 2013",2024,//upload.wikimedia.org/wikipedia/commons/thumb...
98,Wyoming,John Barrasso,"July 21, 1952",Republican,#E81B23,"[Orthopedic surgeon, Medical chief of staff, N...",Wyoming Senate,Rensselaer Polytechnic UniversityGeorgetown Un...,"June 25, 2007[y]",2024,//upload.wikimedia.org/wikipedia/commons/thumb...
